<a href="https://colab.research.google.com/github/sowmya547095/FMML-2023_PROJECTS_AND_ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Question no 1:

Averaging the validation accuracy across multiple splits can indeed provide more consistent and reliable results when evaluating the performance of a machine learning model, especially in scenarios where randomness or variability can impact the training and evaluation process. This practice is commonly referred to as "cross-validation."

Cross-validation involves splitting your dataset into multiple subsets or "folds," training the model on different combinations of these folds, and then averaging the evaluation metrics (such as accuracy) obtained from each fold. There are several benefits to using cross-validation:

Reduced Variance: By averaging the results from multiple folds, you reduce the impact of random variation that might occur when using a single validation set. This provides a more stable and representative estimate of your model's performance.

Better Generalization: Cross-validation helps you assess how well your model generalizes to unseen data, as it evaluates the model's performance on multiple different subsets of the data.

Robustness to Data Splitting: Since you use different data splits for each fold, cross-validation ensures that your evaluation results are less dependent on a specific random split of the data. This makes your model assessment more robust.

Question no 2:

Cross-validation provides a more accurate estimate of how well your machine learning model is likely to perform on unseen data compared to a single train-test split. However, it's essential to understand that cross-validation estimates are not the same as the actual test accuracy you would obtain when deploying your model in a real-world setting.

Here's how cross-validation relates to estimating test accuracy:

More Reliable Estimate: Cross-validation provides a more reliable estimate of a model's performance because it averages results from multiple train-validation splits. This can help reduce the impact of randomness in a single data split, making your estimate more stable and representative.

Closer to True Performance: Cross-validation results are typically closer to the model's true performance on unseen data than a single train-test split. It gives you a better understanding of how well your model generalizes to different parts of the dataset.

Model Selection and Hyperparameter Tuning: Cross-validation is especially valuable when comparing multiple models or tuning hyperparameters. It allows you to make more informed decisions about which model or parameter settings are likely to perform better on unseen data.

Question no 3:

The number of iterations (also known as "folds") in cross-validation can have an effect on the accuracy of the estimate. In general, using a higher number of iterations can provide a better estimate of a model's performance, but there are trade-offs to consider.

Here's how the number of iterations affects the estimate in cross-validation:

More Iterations, Better Estimate: Increasing the number of iterations in cross-validation (e.g., going from 3-fold to 5-fold or 10-fold) typically provides a more accurate and reliable estimate of a model's performance. This is because you are using more diverse subsets of your data for training and validation, reducing the impact of randomness and variability.

Computational Cost: While increasing the number of iterations can improve the estimate's accuracy, it also increases the computational cost. Each additional fold requires training and evaluating the model multiple times, which can be time-consuming, especially for large datasets or complex models.

Diminishing Returns: There is a point of diminishing returns when it comes to the number of iterations. Going from a 3-fold cross-validation to a 10-fold cross-validation might improve the estimate, but the improvement may be relatively small compared to the increase in computational resources required.

Bias-Variance Trade-off: In very small datasets, using a high number of folds can lead to high variance in the estimates because each fold contains very little data. Conversely, in very large datasets, you can often use more folds without significantly increasing the computational cost, leading to more stable estimates.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Question no 1:

Increasing the number of iterations (folds) in cross-validation can help mitigate the challenges of working with a very small training dataset or validation dataset to some extent, but it may not completely solve the underlying issues. Here's how increasing iterations can impact small datasets:

Reduced Variance: With more iterations, you get multiple chances to train and validate your model on different subsets of your small dataset. This can help reduce the impact of randomness in the data split and provide a more stable estimate of model performance.

Increased Robustness: Increasing iterations can make your model assessment more robust by ensuring that you explore a larger portion of your small dataset. It can give you a better understanding of how well your model generalizes, even when working with limited data.

However, there are limitations to consider:

Data Size: Even with more iterations, the fundamental limitation of a small dataset remains. If your dataset is extremely small, it may still lack the diversity and representativeness needed to train a robust machine learning model. Cross-validation can't create more data; it can only make the best use of the available data.

Computational Cost: As you increase the number of iterations, you incur a higher computational cost. This can become a significant concern, especially when working with limited computational resources.

Risk of Overfitting: With very small datasets, there's an increased risk of overfitting during model training, and cross-validation may not completely alleviate this issue. Overfitting occurs when a model learns to fit noise in the data because there isn't enough data to learn the underlying patterns.

Model Choice: The choice of the machine learning algorithm and model complexity becomes crucial when dealing with small datasets. Some models are more robust to small datasets than others, and feature selection or engineering may be necessary to reduce dimensionality and improve performance.

Question no 2:

The size of the validation set in machine learning has a significant impact on the training and evaluation of models. Here's how the accuracy of the validation set is affected when you increase or decrease the percentage of the validation set:

Increasing the Percentage of Validation Set:

Pros:

Better Model Evaluation: A larger validation set provides a more reliable estimate of the model's performance on unseen data. It helps ensure that your model generalizes well to new, unseen examples.
Reduced Overfitting: With a larger validation set, it becomes harder for a model to overfit the validation data. Overfitting occurs when a model learns to perform well on the validation set but doesn't generalize well to new data. A larger validation set helps detect overfitting early.
Cons:

Reduced Training Data: By allocating more data to the validation set, you have less data available for training your model. This can be a problem, especially if you have limited data to begin with. Insufficient training data can lead to underfitting.
Impact on Validation Accuracy: Initially, as you increase the percentage of the validation set, the validation accuracy might improve because you have a better estimate of the model's performance. However, if you allocate too much data to the validation set, it can lead to a decrease in validation accuracy due to the reduced amount of data available for training.

Reducing the Percentage of Validation Set:

Pros:

More Training Data: With a smaller validation set, you have more data available for training your model. This can be beneficial, especially when you have limited data.
Cons:

Less Reliable Model Evaluation: A smaller validation set may not provide an accurate estimate of the model's performance on unseen data. It can lead to higher variance in your evaluation metrics.
Impact on Validation Accuracy: Initially, as you reduce the percentage of the validation set, the validation accuracy may increase because you have more training data. However, if the validation set becomes too small, the accuracy might decrease due to the lack of reliable evaluation data

Question no 3:

The size of the training and validation sets can indeed affect how well you can predict the accuracy on the test set using the validation set. Here's how different scenarios might play out:

Large Training Set, Small Validation Set:

If you have a large training set and a small validation set, you might achieve high training accuracy because the model has more data to learn from. However, the validation accuracy might not be a reliable predictor of test set accuracy. This is because the small validation set may not adequately represent the diversity of the test set, leading to overfitting or underfitting.
Small Training Set, Large Validation Set:

With a small training set and a large validation set, you may have a more accurate estimate of how the model generalizes to new data. However, your model might not perform well on the validation set because it didn't have enough training data to learn meaningful patterns. Consequently, while the validation accuracy is more indicative of test set performance, it might be lower due to the limited training data.
Balanced Training and Validation Sets:

Ideally, you want a balance between the training and validation set sizes. This allows the model to learn meaningful patterns from a sufficient amount of training data while also providing a representative validation set for accurate evaluation. In such cases, the validation accuracy can serve as a reasonably good predictor of the test set accuracy.
Cross-Validation:

To obtain a more robust estimate of model performance, you can use techniques like k-fold cross-validation. This involves dividing your data into multiple folds (typically 5 or 10), training and validating the model on different subsets of the data, and then averaging the validation scores. Cross-validation helps reduce the impact of the specific split of data into training and validation sets and provides a more reliable estimate of test set performance.

Question no 4:

The choice of the percentage to reserve for the validation set can depend on several factors, including the size of your dataset, the complexity of your model, and your specific goals. There is no one-size-fits-all answer, but a common rule of thumb is to allocate around 20-30% of your data to the validation set. However, this is a general guideline and may need to be adjusted based on your circumstances:

Small Datasets: If you have a small dataset (e.g., a few hundred or fewer examples), you might need to allocate a larger percentage (e.g., 30% or more) to the validation set to ensure you have a sufficiently representative sample for evaluation. With limited data, you want to be cautious about overfitting, so a larger validation set can help.

Large Datasets: For larger datasets (e.g., thousands of examples or more), you can often get away with a smaller validation set percentage (e.g., 20%). With more data available for training, you may not need as large a validation set to get a reliable estimate of model performance.

Complex Models: If you are using a very complex model (e.g., deep neural networks with millions of parameters), it's generally a good practice to have a larger validation set to guard against overfitting. In such cases, you might allocate more data to validation.

Cross-Validation: In some cases, especially when you have limited data, you might choose to use cross-validation techniques like k-fold cross-validation. This involves splitting your data into multiple subsets and performing validation and training iterations multiple times. Cross-validation provides a more robust estimate of model performance and can mitigate the impact of the specific validation set size.

Domain Knowledge: Consider domain-specific knowledge and the characteristics of your data. For example, if your data is inherently noisy or has imbalanced classes, you might need a larger validation set to ensure robust model evaluation.